# Fraud Detection Project

## Previous Notebooks

- [EDA](1-EDA.ipynb)

In [1]:
import numpy as np
import pandas as pd
import networkx as nx

In this notebook I will combine the number of cycles calculated in the previous notebook and the degree centrality of each node to assign a score to the parties involved in a claim; then I will use this scores to calculate an overall score for a claim.

The algorithm I will use to assign a score to the nodes of the network is a version of [Google's PageRank](https://en.wikipedia.org/wiki/PageRank) for undirected graphs.

In [2]:
network_raw = pd.read_csv('../data/raw/network.csv', sep=';')

In [3]:
network = network_raw.loc[network_raw['insured_counterpart']=='A'].copy()\
                    .merge(network_raw.loc[network_raw['insured_counterpart']=='C'],
                           how='inner', left_on='claim_code', right_on='claim_code', suffixes=['_lt', '_rt'])

network.drop(['ranking_lt', 'ranking_rt'], axis=1, inplace=True)

In [4]:
nw = nx.MultiGraph()
nw.add_edges_from(network[['id_lt', 'id_rt']].values);

In [5]:
all_cycles = pd.read_pickle('../data/interim/cycles.pkl')

## PageRank

This is how you do PageRank on an undirected graph: given a weight vector $v$ such that $\mathbf{1}^T v = 1$, let $M$ be the adjacency matrix of the graph normalized along the rows (i.e. the sum of the entries of each row is one); then the ranking $r$ is given by

$$r = (1 - d)(I -dM^T)^{-1}v,$$

where $d$ is a real between 0 and 1 called the damping factor.

Since determining the inverse of a big matrix (even a sparse one such as mine) is really expensive I will use an iterative version of the algorithm, namely at each iteration $i$ you have:

$$R(i + 1) = dM^TR(i) + (1 - d)v,$$

where the computation stops when $\left|R(i+1) - R(i)\right| < \epsilon$ for some small $\epsilon$ and $R(0)$ is initialized with $1 / N$ for each node with $N$ the number of nodes in the graph.

In [6]:
from scipy import sparse
from sklearn.preprocessing import normalize

In [7]:
adj = nx.adj_matrix(nw, weight=None)
M = normalize(adj, norm='l1', axis=1)

The choice of the weight vector is really important: if you start with a vector containing just $1/N$s, with $N$ the number of nodes, you just end up with the degree centrality.

In my case I want to consider both the centrality of a node and the number of cycles it is on, so I'm initializing the weights with the number of incident edges plus the number of cycles calculated previously.

In [8]:
v = np.array(list(dict(nx.degree(nw)).values())) + np.array(all_cycles['n_cycles'])
v = v / sum(v)
v = v.reshape(-1, 1)

In [9]:
# initializing the rank with 1/number of nodes
r = np.ones(len(v)) / len(v)
r = r.reshape(-1, 1)

In [10]:
# precision level
eps = 1e-9
d = 0.9 # damping factor
r_prev = r
r = d * M.T.dot(r_prev) + (1-d) * v
while np.linalg.norm(r - r_prev) > eps:
    r_prev = r
    r = d * M.T.dot(r_prev) + (1-d) * v

In [11]:
node_scores = pd.DataFrame(np.hstack([v, r]), index=list(all_cycles['id']))
node_scores.rename(columns={0: 'weights', 1:'pagerank'}, inplace=True)

This is the ranking obtained: adding the number of cycles doesn't change too much the scores in absolute value (but keep in mind that all the scores sum to 1, so I'm dealing with small numbers) but it makes change of position some of the nodes.

In [12]:
node_scores.to_pickle('../data/interim/node_scores.pkl')
node_scores.sort_values(by='pagerank', ascending=False).head(25)

,weights,pagerank
114287,0.000763,0.000928
147731,0.000763,0.000928
96784,0.000716,0.000668
152750,0.000716,0.000668
44886,0.000428,0.000382
112960,0.000428,0.000382
47134,0.000428,0.000382
110762,0.000428,0.000382
69735,0.000428,0.000382
78104,0.000163,0.000157


## Scoring the Claims

Now I'm going to assign a score to each claim: since I could have more than two parties per claim I'm going to use the mean of the scores of all the people involved.

In [13]:
network_raw = network_raw.merge(node_scores, how='left', left_on='id', right_index=True).drop(['weights', 'ranking'], axis=1)

In [14]:
network_raw.head(30)

,id,role,insured_counterpart,claim_code,pagerank
0,145052,PRCO,C,2017075430003700,0.000005
1,144228,PROP,C,2017075430005500,0.000005
2,95994,PROP,A,2017075430007100,0.000005
3,29986,PROP,A,2017075430007700,0.000005
4,95897,PRCO,C,2017075430010100,0.000005
5,95810,PROP,C,2017075430011300,0.000005
6,150043,PRCO,A,2017075430012500,0.000009
7,150043,PROP,A,2017075430012700,0.000009
8,27263,PROP,A,2017075430013100,0.000005
9,25930,PRCO,C,2017075430013100,0.000005


In [15]:
network_raw['score'] = network_raw['pagerank'].fillna(0)

In [16]:
nw_scores = network_raw.groupby('claim_code')['score'].mean().reset_index()
nw_scores.to_pickle('../data/interim/network_scores.pkl')

Finally, let's see how this scores compare to the company ones:

In [17]:
assessments = pd.read_csv('../data/raw/antifraud_assessments.csv', sep=';')

In [18]:
nw_scores = nw_scores.merge(assessments, how='left', left_on='claim_code', right_on='claim_code')

In [19]:
nw_scores.sort_values(by='score', ascending=False).head(50)

,claim_code,score,company_score,ivass_score,fraud_evaluation
84257,2016010402355600,0.000632,47.0,0.0,Frode probabile
23263,2013010000261000,0.000157,23.0,40.0,NaN
120122,2017030430027100,0.000149,61.0,80.0,NaN
118926,2017030400153200,0.000149,79.0,60.0,Frode acclarata
6299,2011010000567500,0.000116,67.0,40.0,Frode acclarata
4700,2011010000197100,0.000116,64.0,40.0,Frode acclarata
21640,2013010000002600,0.000116,64.0,40.0,Frode acclarata
21694,2013010000016800,0.000116,67.0,40.0,Frode acclarata
5874,2011010000481900,0.000116,62.0,60.0,Frode acclarata
12488,2012010000157800,0.000116,71.0,60.0,Frode acclarata


The first 50 claims by PageRank have some unscored claims, some claims with a low score but also a fair number of highly scored claims; not too bad!

Let's also create a crosstab between the company scores and PageRank using the 99th percentile for each of them to classify a claim as a fraud:

In [20]:
print('PageRank 99th percentile: {:.6f}'.format(nw_scores['score'].quantile(0.99)))
print('Company score 99th percentile: {:.6f}'.format(nw_scores['company_score'].dropna().quantile(0.99)))

PageRank 99th percentile: 0.000026
Company score 99th percentile: 58.000000


In [21]:
nw_scores['score_99'] = nw_scores['score'] >= 0.000026
nw_scores['company_score_99'] = nw_scores['company_score'] >= 58.0

In [22]:
pd.crosstab(nw_scores[['score_99', 'company_score_99', 'company_score']].dropna()['score_99'],
            nw_scores[['score_99', 'company_score_99', 'company_score']].dropna()['company_score_99'])

company_score_99,False,True
score_99,,
False,91818,731
True,786,235


I definitely can do better, so after calculating a score also for lawyers and witnesses involved in a claim I'm going to use these PageRanks, alongside with other more standard features, in order to predict if a claim is a fraud.

## Following Notebooks

- [Lawyers' Network Analysis](2.2-Network-Lawyers.ipynb)
- [Witnesses' Network Analysis](2.3-Network-Witnesses.ipynb)
- [Dataset Creation](3-Input_dataset_creation.ipynb)
- [Random Forest Prediction](4-Model.ipynb)